In [5]:
import xarray as xr
xr.set_options(display_style='html')
#import intake
import matplotlib.pyplot as plt
import numpy as np
import cftime
import tqdm
from scipy.stats import linregress
import dask
import pandas as pd
import glob

#from dask.distributed import Client

In [10]:
experiment = "ssp370"
dirs = []
var_path = "Amon/tas"

for x in glob.glob('/badc/cmip6/data/CMIP6/ScenarioMIP/*/*/{e}/*/{v}/*/latest/'.format(e=experiment, v=var_path)):
    dirs.append(x)


In [11]:
len(dirs)

171

In [ ]:


cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)


cat = col.search( experiment_id=['historical','ssp370'],#1pctCO2
             table_id=['Amon'],
             variable_id=['tas'],
             grid_label=['gr'],
            )

dask.config.set(**{'array.slicing.split_large_chunks': True})

dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})
dataset_list = list(dset_dict.keys())



matches = []

for i in dataset_list:
    # print(i)
    
    if '370' in i:
        
        model_config = '.'.join(i.split('.')[1:3])
        
        print(model_config)
        
        x = []
        
        for j in dataset_list:
            
            if model_config in j:
                
                x.append(j)
              
        matches.append(sorted(x))


output = {}

for k in tqdm.tqdm(matches[:-1]):
    
    print(k)
    
    # try:
        
    a = []
    w = []
    y = []

    for i in k:

        data = dset_dict[i]

        winter_mask = data.time.dt.month.isin([12,1,2])
        jan_mask = data.time.dt.month.isin([1]) 
        name = " ".join(i.split('.')[1:3])

        at_data = data['tas'].isel(member_id=0)

        at_data = at_data[winter_mask]

        years = data.time.dt.year[jan_mask].compute()

        # month_length = data.time.dt.days_in_month
        # weights = ( month_length.groupby("time.season") / month_length.groupby("time.season").mean())
        world_annual = (at_data).groupby("time.year").mean(dim="time")

        arctic_annual = world_annual.sel(lat=slice(-90,66))

        world_w = world_annual.weighted(weights=np.cos(np.deg2rad(world_annual.lat)))
        arctic_w = arctic_annual.weighted(weights=np.cos(np.deg2rad(arctic_annual.lat)))

        w += list(world_w.mean(("lon","lat")).compute())
        a += list(arctic_w.mean(("lon","lat")).compute())
        y += list(years)

    output[name] = {'arctic':a,
                    'world':w,
                    'year':y,
                   }
        
#         break
        
#     except:
#         pass

df.set_index('year',inplace=True)
for name, d in output.items():
    
    try:

        df = pd.DataFrame(d)
        
        df.set_index('year',inplace=True)
        
        df.sort_index(inplace=True)
        
        plt.plot(df.index,np.convolve(df['arctic']-np.nanmean(df['arctic'][df.index<1900]),np.ones(9)/9,mode='same'))
        plt.plot(df.index,np.convolve(df['world']-np.nanmean(df['world'][df.index<1900]),np.ones(9)/9,mode='same'))
        
        plt.axhline(1.5)
        
        plt.xlim(1850,2050)
        
        # plt.show()
        
        # break
    
    except:
        print(name)
